<a href="https://colab.research.google.com/github/Triansh/bert/blob/main/MLM_BERT_during_train_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries
* Installing the transformers library 
* Importing the required libraries

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 44.7 MB/s 
     |████████████████████████████████| 895 kB 47.4 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
     |████████████████████████████████| 59 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import random
import gc
import torch.nn as nn
import torch.optim as opt
import re

from torch import backends, manual_seed, rand, flatten, cuda, tensor, long
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForMaskedLM
from transformers import AdamW
from tqdm import tqdm

## Loading Data for pretraining

In [ ]:
!wget --no-check-certificate https://iiitaphyd-my.sharepoint.com/:u:/g/personal/triansh_sharma_students_iiit_ac_in/EXCB2Kzp0xdLjjJhx-gtdzcBfWh2wC59YjNTyI8vbna9Vw\?download\=1

--2021-11-15 05:22:49--  https://iiitaphyd-my.sharepoint.com/:u:/g/personal/triansh_sharma_students_iiit_ac_in/EXCB2Kzp0xdLjjJhx-gtdzcBfWh2wC59YjNTyI8vbna9Vw?download=1
Resolving iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)... 13.107.136.9, 13.107.138.9
Connecting to iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/triansh_sharma_students_iiit_ac_in/Documents/model.tar.gz [following]
--2021-11-15 05:22:49--  https://iiitaphyd-my.sharepoint.com/personal/triansh_sharma_students_iiit_ac_in/Documents/model.tar.gz
Reusing existing connection to iiitaphyd-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 405360556 (387M) [application/x-gzip]
Saving to: ‘EXCB2Kzp0xdLjjJhx-gtdzcBfWh2wC59YjNTyI8vbna9Vw?download=1’

EXCB2Kzp0xdLjjJhx-g 100%[===================>] 386.58M  57.2MB/s    in 9.5s    

2021-11-15 05:22:59 (40.5 MB/s) - ‘EXCB2Kzp0xdLjjJh

In [ ]:
!tar -xf EXCB2Kzp0xdLjjJhx-gtdzcBfWh2wC59YjNTyI8vbna9Vw?download=1

In [ ]:
!wget --no-check-certificate https://iiitaphyd-my.sharepoint.com/:u:/g/personal/triansh_sharma_students_iiit_ac_in/ERHwOs4-tz1Fk4eCst0dKA8BhsU_pxCM7bprY9QY4XpRjQ\?e\=YRl69D\&download\=1

--2021-11-15 05:23:05--  https://iiitaphyd-my.sharepoint.com/:u:/g/personal/triansh_sharma_students_iiit_ac_in/ERHwOs4-tz1Fk4eCst0dKA8BhsU_pxCM7bprY9QY4XpRjQ?e=YRl69D&download=1
Resolving iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)... 13.107.136.9, 13.107.138.9
Connecting to iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/triansh_sharma_students_iiit_ac_in/Documents/wiki.train.zip [following]
--2021-11-15 05:23:06--  https://iiitaphyd-my.sharepoint.com/personal/triansh_sharma_students_iiit_ac_in/Documents/wiki.train.zip
Reusing existing connection to iiitaphyd-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 189564305 (181M) [application/x-zip-compressed]
Saving to: ‘ERHwOs4-tz1Fk4eCst0dKA8BhsU_pxCM7bprY9QY4XpRjQ?e=YRl69D&download=1’

ERHwOs4-tz1Fk4eCst0 100%[===================>] 180.78M  36.5MB/s    in 7.7s    

2021-11-15 05:23:14

In [ ]:
!ls

 data
 dev.tsv
'ERHwOs4-tz1Fk4eCst0dKA8BhsU_pxCM7bprY9QY4XpRjQ?e=YRl69D&download=1'
'EXCB2Kzp0xdLjjJhx-gtdzcBfWh2wC59YjNTyI8vbna9Vw?download=1'
 sample_data
 test.tsv
 train.tsv


In [ ]:
!unzip ERHwOs4-tz1Fk4eCst0dKA8BhsU_pxCM7bprY9QY4XpRjQ\?e=YRl69D\&download=1

Archive:  ERHwOs4-tz1Fk4eCst0dKA8BhsU_pxCM7bprY9QY4XpRjQ?e=YRl69D&download=1
  inflating: wiki.train.tokens       


In [ ]:
with open('./wiki.train.tokens', 'r') as f:
  texts = [ line.strip() for line in f if line.strip() != '' and '=' not in line ][120000:300000]
text = []
reg = re.compile(r"\.|\?|\!")
for t in texts:
  text.extend(reg.split(t))
len(text)

1049946

## Setting  Random SEED and Devices

In [ ]:
SEED = 12345

random.seed(SEED)
manual_seed(SEED)
cuda.manual_seed(SEED)
backends.cudnn.deterministic = True

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
# device = 'cpu'
device

'cuda'

## MLM

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from torch import load
model = load('./data/model')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(
                        text,                        # Sentence to encode.
                        add_special_tokens = True,    # Add '[CLS]' and '[SEP]'
                        max_length = 32,     # Pad & truncate all sentences.
                        padding = 'max_length',
                        truncation=True,
                        return_attention_mask = True, # Construct attn. masks.
                        return_tensors = 'pt',        # Return pytorch tensors.
                    )

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
# create mask array
# Now sentence have a [MASK] with 15 % probability
# Donot mask [PAD], [CLS], [SEP]
randoms = rand(inputs.input_ids.shape)
mask_array = (randoms < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(flatten(mask_array[i].nonzero()).tolist())

In [ ]:
# mark true indexes as [MASK]
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
class Wiki103Datset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = Wiki103Datset(inputs)

In [ ]:
loader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
gc.collect()
cuda.empty_cache()

In [ ]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
optimizer_1 = AdamW(model.parameters(), lr=2e-6)

In [ ]:
epochs = 1

for epoch in range(epochs):
    model.train()
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optimizer_1.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optimizer_1.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/8203 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 0:  42%|████▏     | 3475/8203 [1:29:16<1:59:52,  1.52s/it, loss=0.21]

In [ ]:
from torch import save
save(model, './data2/model')

In [ ]:
!tar -czvf model.tar.gz data2/

In [ ]:
from google.colab import files
files.download('model.tar.gz')

## Loading training and validation data for fine-tuning

In [ ]:
def read_data(file_path):
  return pd.read_csv(file_path, sep='\t', header=None, names=['query', 'rating'])

In [ ]:
train_df =  read_data('./train.tsv')
dev_df = read_data('./dev.tsv')
train_df= train_df.append(dev_df).reset_index(drop=True)
train_df.info()

In [ ]:
validation_df = read_data('./test.tsv')
validation_df.info()

## Dataset Creation
* Creates a QWF (Query well-formedness Dataset) inherited from the pytorch's dataset

In [ ]:
class QWFDataset(Dataset):
  
  def __init__(self, tokenizer, df, maxlen):
    self.df = df.copy()
    self.tokenizer = tokenizer
    self.maxlen = maxlen

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    query = self.df['query'][index]
    label = 1 if self.df['rating'][index] >= 0.8 else 0

    encoded_dict = self.tokenizer.encode_plus(
                        query,                        # Sentence to encode.
                        add_special_tokens = True,    # Add '[CLS]' and '[SEP]'
                        max_length = self.maxlen,              # Pad & truncate all sentences.
                        padding = 'max_length',
                        truncation=True,
                        return_attention_mask = True, # Construct attn. masks.
                        return_tensors = 'pt',        # Return pytorch tensors.
                    )
    ids = encoded_dict['input_ids'].detach().clone().reshape((-1,))
    mask = encoded_dict['attention_mask'].detach().clone().reshape((-1,))

    # print("Shape of ids: ", ids.shape, mask.shape)

    val = {
        'ids': ids,
        'mask': mask,
        'targets': tensor(label, dtype=long)
    } 
    # print(val)
    return val


In [ ]:
TRAIN_BATCH_SIZE = 32
VALIDATION_BATCH_SIZE = 8
MAX_LEN = 32

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 1
                }

validation_params = {'batch_size': VALIDATION_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 1
                }

In [ ]:
print(f"Training Dataset: {train_df.shape}")
print(f"VALIDATION Dataset: {validation_df.shape}")

train_set =  QWFDataset(tokenizer, train_df, maxlen=MAX_LEN)
validation_set =  QWFDataset(tokenizer, validation_df, maxlen=MAX_LEN)

In [ ]:
train_loader = DataLoader(train_set, **train_params)
validation_loader = DataLoader(validation_set, **validation_params)

In [ ]:
class QWFClassificationModel(nn.Module):

  def __init__(self, model):
    super(QWFClassificationModel, self).__init__()

    self.bert = model
    self.pre_classifier = nn.Linear(768, 512)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.3)
    self.classifier = nn.Linear(512, 1)

  def forward(self, tokens, attention_mask):
    output = self.bert(tokens, attention_mask = attention_mask, output_hidden_states=True, return_dict=True)
    hidden_state = output['hidden_states'][0]
    pooler = hidden_state[:, 0]
    pooler = self.pre_classifier(pooler)
    pooler = self.relu(pooler)
    pooler = self.dropout(pooler)
    outputs = self.classifier(pooler)
    return outputs

In [ ]:
from torch import sigmoid
def get_correct(outputs, targets):
  probs = sigmoid(outputs)
  labels = (probs >= 0.5).long()
  correct = (labels==targets).sum().item()
  return correct

In [ ]:
# gc.collect()
# cuda.empty_cache()

In [ ]:
# from torch import load
# model = load('./bt-0.1')

In [ ]:
finetuning_model = QWFClassificationModel(model)
finetuning_model.to(device)

In [ ]:
learning_rate = 1e-5
criterion = nn.BCEWithLogitsLoss()
optimizer = opt.Adam(params = finetuning_model.parameters(), lr = learning_rate)
epochs = 20

In [ ]:
def train(epoch, data_loader):

  training_loss = 0
  training_count = 0
  total_correct = 0

  finetuning_model.train()

  loop = tqdm(data_loader,leave=True)

  for batch in loop:

    optimizer.zero_grad()

    ids = batch['ids'].to(device, dtype = long)
    mask = batch['mask'].to(device, dtype = long)
    targets = batch['targets'].to(device, dtype = long)

    outputs = finetuning_model(ids, mask).squeeze()
    loss = criterion(outputs, targets.float())

    training_loss += loss.item()
    total_correct += get_correct(outputs.data, targets)
    training_count+=targets.size(0)

    loss.backward()
    optimizer.step()

    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())

  epoch_loss = training_loss / len(data_loader)
  epoch_accu = total_correct * 100 / training_count
  print(f"The Mean loss for Epoch: {epoch_loss}")
  print(f"The Total Accuracy for Epoch {epoch}: {epoch_accu}")

  return 

In [ ]:
for epoch in range(epochs):
  train(epoch, train_loader)

In [ ]:
from torch import no_grad

def validate(model, validation_loader):
  finetuning_model.eval()

  validation_loss = 0
  validation_count = 0
  total_correct = 0

  with no_grad():
    loop = tqdm(validation_loader,leave=True)

    for batch in loop:

      ids = batch['ids'].to(device, dtype = long)
      mask = batch['mask'].to(device, dtype = long)
      targets = batch['targets'].to(device, dtype = long)

      outputs = finetuning_model(ids, mask).squeeze()

      loss = criterion(outputs, targets.float())
      validation_loss += loss.item()

      total_correct += get_correct(outputs.data, targets)

      validation_count += targets.size(0)

    accuracy = total_correct * 100 / validation_count
    print(f"Validation Loss : {validation_loss / len(validation_loader)}")
    print(f"Validation Accuracy : {accuracy}")
    return accuracy

In [ ]:
accuracy = validate(model, validation_loader)